Dengan fungsi agg() kita bisa melakukan fungsi aggregate yang berbeda untuk kolom yang berbeda. Pada contoh code dibawah melakukan fungsi mean untuk column sales dan count untuk customer.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("drive/MyDrive/pandas/Data/train.csv", low_memory=False, parse_dates=["Date"])

df.groupby(["Store", "DayOfWeek"]).agg({"Sales": "mean", "Customers": "count"})

Sales  Customers
Store DayOfWeek                        
1     1          4946.119403        134
      2          4650.918519        135
      3          4454.474074        135
      4          4094.607407        135
      5          4516.414815        135
...                      ...        ...
1115  3          5666.155556        135
      4          5396.607407        135
      5          6208.881481        135
      6          6773.388060        134
      7             0.000000        134

[7805 rows x 2 columns]

Atau contoh yang lebih kompleks, menampilkan mean, max dan min untuk kolom sales, dan count untuk kolom customers.

In [4]:
df2 = df.groupby(["Store", "DayOfWeek"]).agg({"Sales": ["mean", "max", "min"], "Customers": "count"})
df2

Sales              Customers
                        mean    max   min     count
Store DayOfWeek                                    
1     1          4946.119403   9528     0       134
      2          4650.918519   7959     0       135
      3          4454.474074   7821     0       135
      4          4094.607407   7785     0       135
      5          4516.414815   8414     0       135
...                      ...    ...   ...       ...
1115  3          5666.155556  10547     0       135
      4          5396.607407  11033     0       135
      5          6208.881481  11673     0       135
      6          6773.388060  10898  4497       134
      7             0.000000      0     0       134

[7805 rows x 4 columns]

Anda juga bisa menggunakan fungsi perhitungan sendiri.

In [5]:
mc_uncert = lambda x: np.std(x) / np.sqrt(x.size)
df2 = df.groupby(["Store", "DayOfWeek"]).agg(
    {"Sales": [("SalesMean", "mean"), ("SalesUncert", mc_uncert)], "Customers": "count"})
df2.head()

Sales             Customers
                   SalesMean SalesUncert     count
Store DayOfWeek                                   
1     1          4946.119403  139.120367       134
      2          4650.918519   94.412423       135
      3          4454.474074   97.225056       135
      4          4094.607407  129.367657       135
      5          4516.414815  113.495466       135

Pendekatan dengan fungsi lambda diatas adalah pendekatan obsolete, walaupun berjalan tanpa masalah. Untuk kedepannya, pendekatan dengan named function lebih disarankan oleh Pandas team.

Berikut hasil refactoring dari code diatas. Bagian header Dataframe yang ditampilkan juga lebih rapih.

In [6]:
def mc_uncert2(x):
    return np.std(x) / np.sqrt(x.size)

dfg = df.groupby(["Store", "DayOfWeek"])
dfg.agg(
    SalesMean=("Sales", "mean"),
    SalesUncert=("Sales", mc_uncert2)
).reset_index().head()

,Store,DayOfWeek,SalesMean,SalesUncert
0,1,1,4946.119403,139.120367
1,1,2,4650.918519,94.412423
2,1,3,4454.474074,97.225056
3,1,4,4094.607407,129.367657
4,1,5,4516.414815,113.495466
